In [1]:
#| hide
from l402.payment_providers import *
from l402.payment_clients import *
from l402.utils import *

# l402-python

> 

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/Fewsats/l402-python.git
```

or from [pypi][pypi]


```sh
$ pip install l402_python
```

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/Fewsats/l402-python
[docs]: https://Fewsats.github.io/l402-python/
[pypi]: https://pypi.org/project/l402-python/
[conda]: https://anaconda.org/Fewsats/l402-python

## How to use

### Server

In [2]:
#| hide
from fastapi.responses import JSONResponse
from http import HTTPStatus
from cdp import *
from fastapi import FastAPI

import httpx


In [3]:
# Create FastAPI app
app = FastAPI()

server = ServerManager(app, port=9000).start()

INFO:     Started server process [53187]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9000 (Press CTRL+C to quit)


## Payment Server

In [4]:
from uuid import uuid4
import json
import os

In [5]:
# w =create_test_wallet(fund=False, chain='base-mainnet')
# import json
# json.dump(w.export_data().to_dict(), open('index-seed.json', 'w'))
# !cat index-seed.json
Cdp.configure(api_key_name=os.getenv("CDP_KEY_NAME"), private_key=os.getenv("CDP_PRIVATE_KEY"))

w = Wallet.import_data(WalletData.from_dict(json.load(open('index-seed.json'))))


In [6]:
w.balance('usdc')

Decimal('0')

In [7]:

ps = PaymentServer(
    payment_request_url="http://localhost:9000/payment_request",
    onchain_provider=w,
)

In [8]:
@app.get("/offers")
def offers():
    offers_list = [Offer(
        amount=1,
        currency='USD',
        description='Purchase 1 credit for API access',
        offer_id=str(uuid4()),
        payment_methods=['onchain'], # maybe not needed because we can generate it from the payment provider
        title='1 Credit Package',
        type='one-time'
    )]

    
    offers_response = ps.create_offers(offers_list)
    return JSONResponse(
        content=offers_response.model_dump(),
        status_code=402
    )

In [9]:
@app.post("/payment_request")
async def create_payment_request(request: PaymentRequest):
    try:
        payment_request = ps.create_payment_request(**request.model_dump())
        return JSONResponse(
            content=payment_request,
            status_code=200
        )
    except Exception as e:
        print(e)

### Client

In [10]:
# w = create_test_wallet(fund=False, chain='base-mainnet')

c = Client(onchain_provider=CoinbaseProvider(wallet=w))

In [11]:
r1 = httpx.get("http://localhost:9000/offers")
r1.status_code, r1.json()

INFO:     127.0.0.1:58674 - "GET /offers HTTP/1.1" 402 Payment Required
INFO:     127.0.0.1:58676 - "POST /payment_request HTTP/1.1" 200 OK
INFO:     127.0.0.1:58699 - "GET /offers HTTP/1.1" 402 Payment Required
INFO:     127.0.0.1:58701 - "POST /payment_request HTTP/1.1" 200 OK
INFO:     127.0.0.1:58703 - "POST /payment_request HTTP/1.1" 200 OK
INFO:     127.0.0.1:58706 - "POST /payment_request HTTP/1.1" 200 OK


(402,
 {'offers': [{'amount': 1,
    'currency': 'USD',
    'description': 'Purchase 1 credit for API access',
    'offer_id': 'aa1d77b4-568d-45c9-aa2d-5608fd7db6e7',
    'payment_methods': ['onchain'],
    'title': '1 Credit Package',
    'type': 'one-time'}],
  'payment_context_token': '74da77bc-0ef6-4125-a592-f8154d23fe3d',
  'payment_request_url': 'http://localhost:9000/payment_request',
  'version': '0.2.2'})

In [12]:
if r1.status_code == HTTPStatus.PAYMENT_REQUIRED:
    try:
        ps = c.pay(r1.json())
    except Exception as e:
        print(e)

Insufficient funds: have 0, need 1.
